# 45 東京の街が底に隠した、水の流れに会いに行く

――暗渠や湧水を地図上に描くと、東京の街が見えてくる

In [ ]:
import requests, json, time, folium; from folium import FeatureGroup; import folium.plugins
from folium_vectorgrid import VectorGridProtobuf; import xml.etree.ElementTree as ET
from pykml import parser; from bs4 import BeautifulSoup
#------------------------------------------------------------------------
def create_folium_map(size): # 東京中心でMapオブジェクトを作る
    m = folium.Map(location=[35.6762, 139.6503], width = size)
    m.get_root().height = size # 縦を指定
    return m
#------------------------------------------------------------------------
def add_overlays(m, overlay_layers): # オーバーレイレイヤー作成してマップに追加する
    for layer in overlay_layers:
        tile_layer = folium.TileLayer(
            attr=layer['attr'], name=layer['name'], tiles=layer['tiles'],
            opacity=1.0, overlay=True, show=False).add_to(m)
    return m
#------------------------------------------------------------------------
def add_river_vector_to_map(m: folium.Map) -> folium.Map:
    river_fg = folium.FeatureGroup( name="水系（河川・湖）［国土地理院ベクトルタイル］",
        attribution="国土地理院 ベクトルタイル提供実験", show=False)
    url = "https://cyberjapandata.gsi.go.jp/xyz/experimental_bvmap/{z}/{x}/{y}.pbf"
    options = { "vectorTileLayerStyles": {
            "river":       {"weight": 5, "color": "#1E90FF", "opacity": 1.0},
            "lake":        {"weight": 2, "color": "#4682B4", "fill": True, 
                           "fillColor": "#4682B4", "fillOpacity": 0.4},
            "railway":     {"weight": 0},
                (中略)
            "wstructurea": {"weight": 0},}}
    vector_grid = VectorGridProtobuf(url=url, options=options, name="river_lake")
    vector_grid.add_to(river_fg); 
    return m
#------------------------------------------------------------------------
def extract_linestrings_from_kml(kml_path): # KML から LineString を読み込む
    with open(kml_path, 'r', encoding='utf-8') as f:
        doc = parser.parse(f).getroot()
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}
    linestrings = []
    for placemark in doc.xpath('.//kml:Document//kml:Placemark', namespaces=ns):
        name = placemark.name.text if hasattr(placemark, 'name') else "無名"
        description = placemark.description.text \
            if hasattr(placemark, 'description') and placemark.description is not None else ""
        if hasattr(placemark, 'LineString'):
            coords = []; coords_text = placemark.LineString.coordinates.text.strip()
            for coord in coords_text.split(): # 座標文字列を [[lat, lon], [lat, lon], ...] に変換
                parts = coord.strip().split(',')
                if len(parts) >= 2:
                    lon = float(parts[0]); lat = float(parts[1])
                    coords.append([lat, lon])  # Foliumは [lat, lon] の順
            if coords:
                linestrings.append({'name':name,'description':description,'coordinates':coords})
    return linestrings
#------------------------------------------------------------------------
def draw_linestrings_on_folium(m,linestrings, 
        layer_name="暗渠", show=False, color="#00CCCC", weight=6, opacity=0.6):
    lines_layer = FeatureGroup(name=layer_name, show=show)
    for line in linestrings:
        folium.PolyLine(
            locations=line['coordinates'], color=color, weight=weight, opacity=opacity,
            popup=folium.Popup( f"<b>{line.get('name', '名称不明')}</b><br>"
                f"{line.get('description', '')}", max_width=300),
            tooltip=line.get('name', 'ライン')
        ).add_to(lines_layer)
    lines_layer.add_to(m)
    return m
#------------------------------------------------------------------------
def get_spring_loc_and_save_geojson(filename):
    URL = "https://www.env.go.jp/water/yusui/result/sub2/tokyo.html"
    GOOGLE_API_KEY = "ここにAPIキーを入れる"
    REQUEST_INTERVAL = 1.0  # Google APIレート制限対策
    # Google Maps ジオコーディング関数
    def geocode_google(query):# GeoJSONを保存eocode_google(query):
        url = ("https://maps.googleapis.com/maps/api/geocode/json"
               f"?address={query}&key={GOOGLE_API_KEY}")
        response = requests.get(url).json()
        if "results" in response and len(response["results"]) > 0:
            location = response["results"][0]["geometry"]["location"]
            return location["lat"], location["lng"], response["results"][0]["formatted_address"]
        return None
    response = requests.get(URL) # 湧水地点リスト抽出
    response.encoding = response.apparent_encoding  # 文字化け対策
    soup = BeautifulSoup(response.text, "html.parser"); rows = soup.find_all("tr")
    geojson = { "type": "FeatureCollection", "features": [] }
    for row in rows: # HTML表データ → ジオコーディング → GeoJSON
        cols = row.find_all("td")
        if len(cols) < 2:
            continue
        name = cols[0].get_text(strip=True); address = cols[2].get_text(strip=True)
        location_name = f"東京都 {address} {name}"
        time.sleep(REQUEST_INTERVAL); result = geocode_google(location_name)
        if result:
            lat, lon, formatted = result
            print(f"   → 緯度経度取得に成功: {lat}, {lon}  ({formatted})")
            geojson["features"].append({
                "type": "Feature", "properties": {"name": name, "address": address,
                "google_formatted": formatted, "source": URL },
                "geometry": {"type": "Point", "coordinates": [lon, lat] } })
        else:
            print("   緯度経度取得に失敗")
    spring_geojson = "tokyo_spring.geojson" # GeoJSONを保存
    with open(output_geojson, "w", encoding="utf-8") as f:
        json.dump(geojson, f, ensure_ascii=False, indent=2)
    return json
#------------------------------------------------------------------------
def read_spring_geojson_and_add_markers(m, filename, layer_name="湧水地", show=False):
    spring_layer = FeatureGroup(name=layer_name, show=show)
    with open(filename, "r", encoding="utf-8") as f:
        geojson = json.load(f)
    for feature in geojson["features"]:
        lon, lat = feature["geometry"]["coordinates"]; props = feature["properties"]
        popup_text = ( f"<b>{props.get('name', '名称不明')}</b><br>"
                       f"{props.get('address', '')}<br>"
                       f"<small>{props.get('google_formatted', '')}</small>")
        folium.Marker( location=[lat, lon], tooltip=props.get('name', '温泉'),
            popup=folium.Popup(popup_text, max_width=300),
            icon=folium.Icon(color="blue", icon="tint", prefix='fa')
        ).add_to(spring_layer)
    spring_layer.add_to(m)
    return m
#------------------------------------------------------------------------
def add_underground_lines(m):
    # https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-N05-v1_3.html　からダウンロードする
    geojson_file = 'N05-19_RailroadSection2.geojson'
    with open(geojson_file, 'r', encoding='utf-8-sig') as f:
        data = json.load(f)
    metro_layer = FeatureGroup(name='東京地下鉄', show=False)
    for feature in data['features']:
        prop = feature['properties']; geom = feature['geometry']; line_name = prop.get('路線名', '')
        if line_name not in ['1号線浅草線', '2号線日比谷線', '3号線銀座線', '4号線丸ノ内線',
            '4号線丸ノ内線分岐線', '5号線東西線', '三田線', '南北線', '8号線有楽町線',
            '9号線千代田線', '新宿線', "半蔵門線", "12号線大江戸線", "13号線副都心線"]:
            continue
        if geom['type'] == 'LineString':
            coords = [[coord[1], coord[0]] for coord in geom['coordinates']]
            folium.PolyLine( locations=coords, color='#FF0000', weight=3.5,
                tooltip=f"{line_name}" ).add_to(metro_layer)
    metro_layer.add_to(m)
    return m

#------実行部------------------------------------------------------------------
overlay_layers = [{'name':'国土地理院 デジタル標高地形図（東京都区部）','attr': '国土地理院',
          'tiles': 'https://cyberjapandata.gsi.go.jp/xyz/d1-no455/{z}/{x}/{y}.png'},
                (中略)
          {'name': '日本シームレス地質図','attr': '出典: 地質調査総合センター',
           'tiles':'https://gbank.gsj.jp/seamless/v2/api/1.2/tiles/{z}/{y}/{x}.png'}]

m = create_folium_map(800); add_overlays(m, overlay_layers)  # マップを作り、ラスタレイヤを追加

add_river_vector_to_map(m)  # 川のベクトルマップをマップに追加する

# c.f. https://thetokyofiles.com/2016/01/17/walking-on-water-the-underground-rivers-of-tokyo/
url = "https://www.google.com/maps/d/u/0/kml?forcekml=1&mid=1iY7aQDtcp_LBP6JZW-DPsV4Zzgk"
kml_file = "tokyo_files.kml"
with open(kml_file, "wb") as f:
    f.write(requests.get(url).content)
lines = extract_linestrings_from_kml(kml_file)　# 暗渠マップを読み込み、マップに追加する
draw_linestrings_on_folium(m, lines)

read_spring_geojson_and_add_markers(m, filename="tokyo_spring.geojson") # 湧水地もマップに追加

add_underground_lines(m) # 地下鉄も追加しておく

# クリックした地点のGoogleStreetViewを眺めるためのJavaScript注入
js_code = """<script>document.addEventListener("DOMContentLoaded", function() {
for (var map_id in window) {
    if (window[map_id] && window[map_id].getCenter) {
        window[map_id].on('click', function(e) {
            var lat = e.latlng.lat.toFixed(6); var lng = e.latlng.lng.toFixed(6);
            var url = "https://www.google.com/maps/@?api=1&map_action=pano&viewpoint="+lat+","+lng;
            window.open(url, '_blank');
        });}}});</script>"""
m.get_root().html.add_child(folium.Element(js_code))
folium.plugins.Geocoder().add_to(m) # 場所検索バーを作り、マップに追加する
folium.LayerControl().add_to(m)     # レイヤーコントロールをマップに追加する
m